In [1]:

from llama_index.core.agent import AgentRunner
from llama_index.llms.vertex import Vertex
from llama_index.core import Settings
from llama_index.core.llms import ChatMessage
from google.oauth2 import service_account
from session import Session
from llama_index.core.storage.chat_store import SimpleChatStore
import time


filename = "../key.json"
credentials: service_account.Credentials = (
    service_account.Credentials.from_service_account_file(filename)
)
llm = Vertex(
    model="gemini-1.5-pro-001", project=credentials.project_id, credentials=credentials
)
#
Settings.llm = llm



In [13]:
from llama_index.core import PromptTemplate

In [3]:
# chat_store = {}

persist_path = "chat_store.json"
chat_store = SimpleChatStore.from_persist_path(persist_path)

agent = AgentRunner.from_llm(llm=llm, tools=[], verbose=True)
agent_prompt = agent.get_prompts()

In [14]:
print(agent_prompt['agent_worker:system_prompt'].template)

You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 'hello world', 'num_beams': 

In [21]:
custom_template = """"You are a supervisor who analyzes live telephone conversations between a human agent and a customer, instantly providing relevant recommendations to the agent. Your assistance helps the agent improve the quality of the interaction and respond more effectively to the customer's needs.

- You listen to the discussions between the customer and the agent.
- You analyze the customer's sentiment.
- You give suggestions to the agent to articulate the call well.
- You alert the agent if you deem it necessary.

## Current Conversation

Below is the current conversation consisting of interleaving customer and agent messages."""

In [22]:
custom_prompt = PromptTemplate(custom_template)

In [23]:
agent.update_prompts(
    {"agent_worker:system_prompt": custom_prompt}
)

In [24]:



def get_response(query, chat_history):

    global agent

    response = agent.chat(query, chat_history=chat_history)
    chat_history = agent.memory.chat_store.store["chat_history"]

    return response, chat_history


def ask(query: str, user_id: str) -> str:
    global chat_store

    session = Session(user_id=user_id, chat_store=chat_store, persist_path=persist_path)

    chat_history = session.get_messages()

    response, chat_history = get_response(query, chat_history)
    session.set_messages(chat_history)

    return response


In [27]:
t1 = time.time()
print("asking: ", t1)
print(ask("C'est qui à l'appareil?", 'user_6'))

t2 = time.time()
print("execution time: ", t2 - t1)


asking:  1720100836.630905
Thought: (Implicit) I can answer without any more tools!
Answer: Okay, the customer is asking "Who is this?" 

I need more context to provide helpful recommendations to the agent.  Please tell me:

* **What was the context of this call? Did the agent call the customer, or did the customer call in?**
* **What is the purpose of the call from the agent's perspective?** 

For example, if the agent placed the call, they should probably introduce themselves and the reason for calling. If the customer called in, the agent should politely confirm their identity and offer assistance. 

Please provide me with more information about the conversation! 

Okay, the customer is asking "Who is this?" 

I need more context to provide helpful recommendations to the agent.  Please tell me:

* **What was the context of this call? Did the agent call the customer, or did the customer call in?**
* **What is the purpose of the call from the agent's perspective?** 

For example, if t

In [31]:
chat_store.store

{'sitraka': [ChatMessage(role=<MessageRole.USER: 'user'>, content='Salut?', additional_kwargs={}),
  ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Salut ! Comment allez-vous ?', additional_kwargs={}),
  ChatMessage(role=<MessageRole.USER: 'user'>, content='Hello what can you help me?', additional_kwargs={}),
  ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='I am a large language model, I can help you with a variety of tasks, from answering questions to providing summaries to other types of analyses. For example, I can provide summaries of factual topics or create stories.', additional_kwargs={})],
 'herival': [ChatMessage(role=<MessageRole.USER: 'user'>, content='Hello what can you help me?', additional_kwargs={}),
  ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='I am a large language model, designed to help with a variety of tasks. I can answer your questions, provide summaries of text, and perform other types of analyses. For example